In [1]:
import pandas as pd
import numpy as np

In [2]:
typed = pd.read_json('./rawData/cleanedTyped.json')

In [3]:
interaction = pd.read_json('./typedData/withInteraction.json')
interaction['price_per_num_lux'] = interaction['price']/interaction['num_luxury']

In [4]:
typeCols = typed.columns
interCols = interaction.columns

In [5]:
interCols = [x for x in interCols if x not in typeCols]

In [6]:
typeCols

Index(['Balcony', 'Cats_Allowed', 'Common_Outdoor_Space', 'Dining_Room',
       'Dishwasher', 'Dogs_Allowed', 'Doorman', 'Elevator', 'Exclusive',
       'Fitness_Center', 'Garden_Patio', 'Hardwood_Floors',
       'High_Speed_Internet', 'Laundry_in_Unit', 'New_Construction', 'No_Fee',
       'Outdoor_Space', 'Pre_War', 'Roof_Deck', 'Swimming_Pool', 'Terrace',
       'Wheelchair_Access', 'apartment', 'bathrooms', 'bedrooms',
       'building_id', 'condominium', 'created', 'description',
       'display_address', 'features', 'foundType', 'interestVal',
       'interest_level', 'latitude', 'laundry_in_building', 'listing_id',
       'loft', 'longitude', 'manager_id', 'other', 'ph', 'price',
       'street_address', 'studio', 'townhome', 'type', 'walk_up'],
      dtype='object')

In [7]:
interCols

['Loft',
 'ada',
 'door_excl',
 'fitness_oriented',
 'lux_per_dollar',
 'lux_score',
 'num_features',
 'num_luxury',
 'outdoor_score',
 'pets_allowed',
 'price_per_num_lux']

In [8]:
m1 = pd.DataFrame(interaction[interCols])
m1.drop('Loft', axis=1, inplace=True)
m1.replace(np.inf, np.nan, inplace=True)

In [9]:
interaction_typed = pd.concat([typed, m1], axis=1)

In [10]:
interaction_typed['price_per_feature'] = interaction_typed['price']/interaction_typed['num_features']

In [11]:
interaction_typed['price_per_feature'].replace(np.inf, np.nan, inplace=True)

In [12]:
g1 = interaction_typed.groupby(['type']).mean()
g1.reset_index(inplace=True)

In [13]:
avgs = g1[['type','lux_score', 'num_features', 'num_luxury','outdoor_score', 'price_per_num_lux', 'price_per_feature']]

In [14]:
avgs.columns = ['avg_'+x for x in avgs]
avgs.rename(columns={'avg_type':'type'}, inplace=True)

/Users/Braden/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [15]:
avgAdded = pd.merge(interaction_typed, avgs, on='type')

I replaced null values with the average to not skew the results. Avaging will result in a ratio of 1 indicating that this property is not differentiated from the rest.

In [16]:
avgAdded['price_per_num_lux'].fillna(avgAdded['avg_price_per_num_lux'], inplace=True)
avgAdded['outdoor_score'].fillna(avgAdded['avg_outdoor_score'], inplace=True)
avgAdded['lux_score'].fillna(avgAdded['avg_lux_score'], inplace=True)
avgAdded['price_per_feature'].fillna(avgAdded['avg_price_per_feature'], inplace=True)

avgAdded['price_lux_ratio'] = avgAdded['price_per_num_lux']/avgAdded['avg_price_per_num_lux']
avgAdded['outdoor_ratio'] = avgAdded['outdoor_score']/avgAdded['avg_outdoor_score']
avgAdded['lux_ratio'] = avgAdded['lux_score']/avgAdded['avg_lux_score']
avgAdded['price_feature_ratio'] = avgAdded['price_per_feature']/avgAdded['avg_price_per_feature']

In [17]:
train = pd.read_json('./rawData/train_data.json')
train['numPhotos'] = train['photos'].apply(len)
avgAdded = avgAdded.merge(train[['numPhotos', 'listing_id']], on='listing_id', how='left')

In [18]:
avgAdded.to_json('./typedData/compsTypedInteraction.json')